### Generate JCM Default Statistics

This notebook is used to generate the statistics (mean and std deviation) used in the model unit test model_test.py:test_speedy_model_default_statistics(). If the model behavior changes meaningfully we will want to rerun these and generate a new default_statistics.nc file.

In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu" # for use on ARM Macs

import jax
print("JAX device:", jax.devices())
jax.config.update('jax_disable_jit', False) # Turn off JIT because of an issue in shortwave_radiation.py:169
jax.config.update("jax_debug_infs", True) # doesn't add any time since the saved time is otherwise spent getting the nodal quantities
jax.config.update("jax_debug_nans", False) # some physics fields might be nan

Metal device set to: Apple M3 Max

systemMemory: 36.00 GB
maxCacheSize: 13.50 GB

JAX device: [CpuDevice(id=0)]


W0000 00:00:1762192746.394679 11105726 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1762192746.406313 11105726 service.cc:145] XLA service 0x11e979090 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762192746.406321 11105726 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1762192746.407394 11105726 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1762192746.407403 11105726 mps_client.cc:384] XLA backend will use up to 28990554112 bytes on device 0 for SimpleAllocator.


In [2]:
from jcm.data.test.t30.generate_default_stats import run_default_speedy_model
model, predictions = run_default_speedy_model()

In [8]:
pred_ds = model.predictions_to_xarray(predictions)
pred_ds_mean = pred_ds.mean(dim='lon').resample(time='1ME').mean() # resample to monthly intervals
pred_ds_std = pred_ds.mean(dim='lon').resample(time='1ME').std() # resample to monthly intervals

#### Save Stats to netCDF

In [11]:
import xarray as xr

q_mean = pred_ds_mean.isel(time=-1).specific_humidity
q_std = pred_ds_std.isel(time=-1).specific_humidity

t_mean = pred_ds_mean.isel(time=-1).temperature
t_std = pred_ds_std.isel(time=-1).temperature

u_mean = pred_ds_mean.isel(time=-1).u_wind
u_std = pred_ds_std.isel(time=-1).u_wind

v_mean = pred_ds_mean.isel(time=-1).v_wind
v_std = pred_ds_std.isel(time=-1).v_wind

nsp_mean = pred_ds_mean.isel(time=-1).normalized_surface_pressure
nsp_std = pred_ds_std.isel(time=-1).normalized_surface_pressure

phi_mean = pred_ds_mean.isel(time=-1).geopotential
phi_std = pred_ds_std.isel(time=-1).geopotential

# total outgoing radiation at top of atmosphere
TOA_rad = pred_ds['shortwave_rad.ftop']-pred_ds['longwave_rad.ftop']
TOA_rad_mean = TOA_rad.mean(dim='lon').resample(time='1ME').mean()
TOA_rad_std = TOA_rad.mean(dim='lon').resample(time='1ME').std()

# relative humidity
rh_mean = pred_ds_mean['humidity.rh'].isel(time=-1)
rh_std = pred_ds_std['humidity.rh'].isel(time=-1)

# stratiform cloud cover
cloudstr_mean = pred_ds_mean['shortwave_rad.cloudstr'].isel(time=-1)
cloudstr_std = pred_ds_std['shortwave_rad.cloudstr'].isel(time=-1)

# equivalent specific humidity of clouds
qcloud_mean = pred_ds_mean['shortwave_rad.qcloud'].isel(time=-1)
qcloud_std = pred_ds_std['shortwave_rad.qcloud'].isel(time=-1)

# precip from convection
precnv_mean = pred_ds_mean['convection.precnv'].isel(time=-1)
precnv_std = pred_ds_std['convection.precnv'].isel(time=-1)

# precip from large scale condensation
precls_mean = pred_ds_mean['condensation.precls'].isel(time=-1)
precls_std = pred_ds_std['condensation.precls'].isel(time=-1)

ds = xr.Dataset({"specific_humidity.mean": q_mean, "specific_humidity.std": q_std,
                "temperature.mean": t_mean, "temperature.std": t_std,
                "u_wind.mean": u_mean, "u_wind.std": u_std,
                "v_wind.mean": v_mean, "v_wind.std": v_std,
                "normalized_surface_pressure.mean": nsp_mean, "normalized_surface_pressure.std": nsp_std,
                "geopotential.mean": phi_mean, "geopotential.std": phi_std,
                "TOA_rad.mean": TOA_rad_mean, "TOA_rad.std": TOA_rad_std,
                "relative_humidity.mean": rh_mean, "relative_humidity.std": rh_std,
                "shortwave_rad.cloudstr.mean": cloudstr_mean, "shortwave_rad.cloudstr.std": cloudstr_std,
                "shortwave_rad.qcloud.mean": qcloud_mean, "shortwave_rad.qcloud.std": qcloud_std,
                "convection.precnv.mean": precnv_mean, "convection.precnv.std": precnv_std,
                "condensation.precls.mean": precls_mean, "condendation.precls.std": precls_std
                })


In [12]:
ds.to_netcdf('jcm/data/test/t30/default_statistics.nc')